# Basic Libraries

In [1]:
%matplotlib inline
import random
import math
import numpy as np
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

import numpy as np

from qiskit.providers.aer.noise import NoiseModel
from qiskit.aqua import QuantumInstance
from qiskit.providers.aer.noise import thermal_relaxation_error, ReadoutError, depolarizing_error

provider = IBMQ.get_provider(hub='ibm-q-education', group = 'rutgers-uni-Schn', project= 'vqe-research')
machine = provider.get_backend('ibmq_casablanca')
# from qiskit.test.mock import FakeLondon
# machine = FakeLondon()
machine_noise_model = NoiseModel.from_backend(machine)
simulator = Aer.get_backend('qasm_simulator')


basis_gates = machine_noise_model.basis_gates

C:\Users\isido\anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
config = machine.configuration()
print("This backend is called {0}, and is on version {1}. It has {2} qubit{3}. It "
      "{4} OpenPulse programs. The basis gates supported on this device are {5}."
      "".format(config.backend_name,
                config.backend_version,
                config.n_qubits,
                '' if config.n_qubits == 1 else 's',
                'supports' if config.open_pulse else 'does not support',
                config.basis_gates))

This backend is called ibmq_casablanca, and is on version 1.0.1. It has 7 qubits. It supports OpenPulse programs. The basis gates supported on this device are ['id', 'u1', 'u2', 'u3', 'cx'].


## Gate Insertion

In [3]:
from GateInsertion_funcs import splitter, PiReader, GateInsertion, u3PairAdder, cXPairAdder, GateInserter

## Pulse Stretching

In [4]:
from newPulseFuncs import instruc_sorter, stretch_sub_sched, stretcher, scheduler

## Result Functions

In [5]:

# Expectation Value/Running Functions
from Result_funcs import binaryToDecimal, dictToList, sampleExpecVal, countToEig, expecValForSamples, addDicts, evaluator

# VQE Functions

In [6]:
#Local Hamiltonian Function
from VQEHamiltonian_funcs import Hammifier

In [7]:
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

initial_state = HartreeFock(
4,
2,
'parity'
) 
var_form = UCCSD(
            num_orbitals=4,
            num_particles=2,
            initial_state=initial_state,
            qubit_mapping='parity'
        )

In [8]:
param = [ 0.00172129,-0.00182397, -0.11217598]
circ = var_form.construct_circuit(param).decompose()

## Stretching and Prepping

In [9]:
scales = [1+0.2*i for i in range(11)]

In [10]:
scales

[1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4000000000000004, 2.6, 2.8, 3.0]

In [11]:
def EnergyFinder(scale):
    '''
    Input: param (1D array) for which to compute energy on VQE circ of H2
    Output: energy
    '''
    
    Hamiltonian = """II\nIZ\nZI\nZZ\nXX"""
    Hamiltonian_weights = [-1.053, 0.395, -0.395, -0.011, 0.181]
    global machine
    backend = machine
    samples = 8192
    copies = 75
    
    pulse_energies = []
    gate_energies = []
    for j in range(copies):
        
        global circ
        circ_ =circ.copy()
        circuits = Hammifier(circ_, Hamiltonian)
        scaled_circuits_Gate = GateInserter(scale, circuits)
        scaled_circuits_Pulse = scheduler(scale, circuits, machine = machine)
        
        gate_energy= evaluator(scaled_circuits_Gate , samples, Hamiltonian, hammy_weights = Hamiltonian_weights, machine= machine)
        pulse_energy= evaluator(scaled_circuits_Pulse , samples, Hamiltonian, hammy_weights = Hamiltonian_weights, machine= machine)
        
        print(gate_energy)
        print(pulse_energy)
        
        pulse_energies.append(pulse_energy)
        gate_energies.append(gate_energy)
        
    return pulse_energies, gate_energies    
    

In [12]:
gate_energiesss = []
pulse_energiesss = []
for c in scales:
    print('--------------' + 'Scale :' + str(c) + ' -------------')
    pulse_energies, gate_energies = EnergyFinder(c)
    gate_energiesss += [gate_energies]
    pulse_energiesss += [pulse_energies]


--------------Scale :1.0 -------------
Job Status: job has successfully run


C:\Users\isido\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\isido\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Job Status: job has successfully run


IndexError: index 33 is out of bounds for axis 0 with size 4